In [1]:
import numpy as np
from scipy.optimize import minimize

# Constraints for a given z
def check_feasibility(z):
    def constraint1(x):
        x1, x2 = x
        return (2 * x1) / (3 * x1 + x2 + 1) - (2**(10 * z) - 1)

    def constraint2(x):
        x1, x2 = x
        return (3 * x2) / (2 * x1 + 2 * x2 + 1) - (2**(12 * z) - 1)
    
    # Bounds for x1 and x2
    bnds = [(0, 1), (0, 1)]
    
    # Initial guess
    x0 = [0.5, 0.5]
    
    # Constraints dictionary
    con1 = {'type': 'ineq', 'fun': constraint1}
    con2 = {'type': 'ineq', 'fun': constraint2}
    cons = [con1, con2]
    
    # Try to find a feasible solution
    solution = minimize(lambda x: 0, x0, method='SLSQP', bounds=bnds, constraints=cons)
    
    return solution.success

# Bisection method to find the maximum z
def bisection_z(z_low, z_high, tol=1e-6):
    while z_high - z_low > tol:
        z_mid = (z_low + z_high) / 2
        if check_feasibility(z_mid):
            z_low = z_mid
        else:
            z_high = z_mid
    return z_low

# Initial range for z
z_low = 0
z_high = 1

# Perform bisection to find the optimal z
optimal_z = bisection_z(z_low, z_high)

print(f"Optimal z: {optimal_z}")


C:\Users\afmb\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\afmb\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\afmb\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Optimal z: 0.050296783447265625


# The following is with Bisection over z:

In [2]:
import numpy as np
from scipy.optimize import minimize

# Constraints for a given z
def check_feasibility(z):
    def constraint1(x):
        x1, x2 = x
        return (2 * x1) / (3 * x1 + x2 + 1) - (2**(10 * z) - 1)

    def constraint2(x):
        x1, x2 = x
        return (3 * x2) / (2 * x1 + 2 * x2 + 1) - (2**(12 * z) - 1)
    
    # Bounds for x1 and x2
    bnds = [(0, 1), (0, 1)]
    
    # Initial guess
    x0 = [0.5, 0.5]
    
    # Constraints dictionary
    con1 = {'type': 'ineq', 'fun': constraint1}
    con2 = {'type': 'ineq', 'fun': constraint2}
    cons = [con1, con2]
    
    # Try to find a feasible solution
    solution = minimize(lambda x: 0, x0, method='SLSQP', bounds=bnds, constraints=cons)
    
    if solution.success:
        return True, solution.x
    else:
        return False, None

# Bisection method to find the maximum z and corresponding x1, x2
def bisection_z(z_low, z_high, tol=1e-6):
    best_x = None
    while z_high - z_low > tol:
        z_mid = (z_low + z_high) / 2
        feasible, x = check_feasibility(z_mid)
        if feasible:
            z_low = z_mid
            best_x = x
        else:
            z_high = z_mid
    return z_low, best_x

# Initial range for z
z_low = 0
z_high = 1

# Perform bisection to find the optimal z
optimal_z, optimal_x = bisection_z(z_low, z_high)

x1_opt, x2_opt = optimal_x
print(f"Optimal z: {optimal_z}")
print(f"Optimal x1: {x1_opt}")
print(f"Optimal x2: {x2_opt}")


Optimal z: 0.050296783447265625
Optimal x1: 0.9999555709565521
Optimal x2: 0.794637088591275


In [2]:
B = np.array([[0.        , 0.00280774, 0.00227045, 0.00599378, 0.00054239],
              [0.00280774, 0.        , 0.00090963, 0.10872945, 0.00079498],
              [0.00227045, 0.00090963, 0.        , 0.00180449, 0.03946987],
              [0.00599378, 0.10872945, 0.00180449, 0.        , 0.00094696],
              [0.00054239, 0.00079498, 0.03946987, 0.00094696, 0.        ]])

a = 2 * np.array([0.038254, 0.37979634, 6.42646204, 1.4801049, 1.43769989])
b = np.array([0.00206423, 0.04449928, 1.3862424, 0.29103015, 0.29611187])
I_M = np.array([7.78643045e-13, 2.45343970e-12, 1.00922029e-11, 4.84335440e-12, 4.77346915e-12])
beta = np.array([8,6,8,2,7])

In [7]:
import numpy as np
from scipy.optimize import minimize
import math


# Number of variables
K = len(a)

# Constraints for a given z
def check_feasibility(z):
    def generate_constraints(z):
        constraints = []
        for j in range(K):
            def constraint(p, j=j):
                return (a[j] * p[j]) / (b[j] * p[j] + np.sum(B[j, :] * p) - B[j, j] * p[j] + I_M[j]) - (2**(beta[j] * z) - 1)
            constraints.append({'type': 'ineq', 'fun': constraint})
        return constraints
    
    # Bounds for p1 to pK
    bnds = [(0, 1)] * K
    
    # Initial guess
    p0 = 1*np.full(K, 0.5)
    
    # Generate constraints based on z
    cons = generate_constraints(z)
    
    # Try to find a feasible solution
    solution = minimize(lambda p: 0, p0, method='SLSQP', bounds=bnds, constraints=cons)
    
    if solution.success:
        return True, solution.x
    else:
        return False, None

# Bisection method to find the maximum z and corresponding p1, ..., pK
def bisection_z(z_low, z_high, tol=1e-6):
    best_p = None
    while z_high - z_low > tol:
        z_mid = (z_low + z_high) / 2
        feasible, p = check_feasibility(z_mid)
        if feasible:
            z_low = z_mid
            best_p = p
        else:
            z_high = z_mid
    return z_low, best_p

# Initial range for z
z_low = 0
z_high = 1

# Perform bisection to find the optimal z
optimal_z, optimal_p = bisection_z(z_low, z_high)

if optimal_p is not None:
    print(f"Optimal z: {optimal_z}")
    print(f"Optimal p: {optimal_p}")
else:
    print("No feasible solution found.")


Optimal z: 0.41864013671875
Optimal p: [0.40067573 0.02233334 1.         0.00321495 0.28774007]


In [6]:
# Function to compute SINR
def compute_SINR(p, a, b, B, I_M):
    K = len(a)
    SINR = np.zeros(K)
    for j in range(K):
        interference = np.sum( B*p ) + b[j] * p[j] + I_M[j]
        SINR[j] = (a[j] * p[j]) / interference
    return SINR

# Function to compute latencies
def compute_latencies(SINR, beta):
    K = len(beta)
    latencies = np.zeros(K)
    for j in range(K):
        if SINR[j] > 0:
            latencies[j] = beta[j] / math.log2(1 + SINR[j])
        else:
            latencies[j] = float('inf')  # Handle cases where SINR[j] is zero
    return latencies

if optimal_p is not None:
    print(f"Optimal z: {optimal_z}")
    print(f"Optimal p: {optimal_p}")
    
    # Compute SINR with optimal_p
    SINR_optimal = compute_SINR(optimal_p, a, b, B, I_M)
    print("\nSINR:")
    for j in range(len(a)):
        print(f"User {j+1}: {SINR_optimal[j]}")
    
    # Compute latencies with SINR_optimal
    latencies_optimal = compute_latencies(SINR_optimal, beta)
    print("\nLatencies:")
    for j in range(len(beta)):
        print(f"User {j+1}: {latencies_optimal[j]}")
else:
    print("No feasible solution found.")


Optimal z: 0.41864013671875
Optimal p: [0.40067573 0.02233334 1.         0.00321495 0.28774007]

SINR:
User 1: 0.4726593629291971
User 2: 0.26089623286813357
User 3: 8.862425970737785
User 4: 0.14649341905238508
User 5: 5.544156875390188

Latencies:
User 1: 14.326037916076624
User 2: 17.939925358893696
User 3: 2.4228162180424313
User 4: 10.140544201889181
User 5: 2.5828282270021536


In [4]:
if optimal_p is not None:
    print(f"Optimal z: {optimal_z}")
    print(f"Optimal p: {optimal_p}")
else:
    print("No feasible solution found.")

Optimal z: 0.41864013671875
Optimal p: [0.09505497 0.00529829 0.23723665 0.0007627  0.06826249]


In [8]:
import numpy as np
from scipy.optimize import minimize
import math

# Given parameters
a = np.array([0.038254, 0.37979634, 6.42646204, 1.4801049, 1.43769989])
b = np.array([0.00206423, 0.04449928, 1.3862424, 0.29103015, 0.29611187])
B = np.array([[0.0, 0.00280774, 0.00227045, 0.00599378, 0.00054239],
              [0.00280774, 0.0, 0.00090963, 0.10872945, 0.00079498],
              [0.00227045, 0.00090963, 0.0, 0.00180449, 0.03946987],
              [0.00599378, 0.10872945, 0.00180449, 0.0, 0.00094696],
              [0.00054239, 0.00079498, 0.03946987, 0.00094696, 0.0]])
I_M = np.array([7.78643045e-13, 2.45343970e-12, 1.00922029e-11, 4.84335440e-12, 4.77346915e-12])
beta = np.array([8, 6, 8, 2, 7])


# Constraints for a given z
def check_feasibility(z):
    def generate_constraints(z):
        constraints = []
        for j in range(K):
            def constraint(p, j=j):
                return (a[j] * p[j]) / (np.sum(B * p) + b[j] * p[j] + I_M[j]) - (2**(beta[j] * z) - 1)
            constraints.append({'type': 'ineq', 'fun': constraint})
        return constraints
    
    # Bounds for p1 to pK
    bnds = [(0, 1)] * K
    
    # Initial guess
    p0 = 0.2 * np.ones(K)
    
    # Generate constraints based on z
    cons = generate_constraints(z)
    
    # Try to find a feasible solution
    solution = minimize(lambda p: 0, p0, method='SLSQP', bounds=bnds, constraints=cons)
    
    if solution.success:
        return True, solution.x
    else:
        return False, None

# Bisection method to find the maximum z and corresponding p1, ..., pK
def bisection_z(z_low, z_high, tol=1e-6):
    best_p = None
    while z_high - z_low > tol:
        z_mid = (z_low + z_high) / 2
        feasible, p = check_feasibility(z_mid)
        if feasible:
            z_low = z_mid
            best_p = p
        else:
            z_high = z_mid
    return z_low, best_p

# Initial range for z
z_low = 0
z_high = 1

# Perform bisection to find the optimal z
optimal_z, optimal_p = bisection_z(z_low, z_high)

if optimal_p is not None:
    print(f"Optimal z: {optimal_z}")
    print(f"Optimal p: {optimal_p}")
    
    # Compute SINR with optimal_p
    SINR_optimal = compute_SINR(optimal_p, a, b, B, I_M)
    print("\nSINR:")
    for j in range(len(a)):
        print(f"User {j+1}: {SINR_optimal[j]}")
    
    # Compute latencies with SINR_optimal
    latencies_optimal = compute_latencies(SINR_optimal, beta)
    print("\nLatencies:")
    for j in range(len(beta)):
        print(f"User {j+1}: {latencies_optimal[j]}")
else:
    print("No feasible solution found.")


Optimal z: 0.17447757720947266
Optimal p: [0.9724283  0.06669958 0.00814544 0.00406377 0.02653985]

SINR:
User 1: 1.6313846534444643
User 2: 1.0660361007422863
User 3: 1.6314134503705886
User 4: 0.273678387793205
User 5: 1.3316366995677085

Latencies:
User 1: 5.731389179379779
User 2: 5.731395495013725
User 3: 5.731324351942019
User 4: 5.730642054695595
User 5: 5.731395615875289


# The following is with scipy:

In [20]:
import numpy as np
from scipy.optimize import minimize

# Constraints for a given z
def check_feasibility(z):
    def constraint1(x):
        x1, x2 = x
        return (2 * x1) / (3 * x1 + x2 + 1) - (2**(180 * z) - 1)

    def constraint2(x):
        x1, x2 = x
        return (3 * x2) / (2 * x1 + 2 * x2 + 1) - (2**(100 * z) - 1)
    
    # Bounds for x1 and x2
    bnds = [(0, 1), (0, 1)]
    
    # Initial guess
    x0 = [0.5, 0.5]
    
    # Constraints dictionary
    con1 = {'type': 'ineq', 'fun': constraint1}
    con2 = {'type': 'ineq', 'fun': constraint2}
    cons = [con1, con2]
    
    # Try to find a feasible solution
    solution = minimize(lambda x: 0, x0, method='SLSQP', bounds=bnds, constraints=cons)
    
    if solution.success:
        return True, solution.x
    else:
        return False, None

# Bisection method to find the maximum z and corresponding x1, x2
def bisection_z(z_low, z_high, tol=1e-6):
    best_x = None
    while z_high - z_low > tol:
        z_mid = (z_low + z_high) / 2
        feasible, x = check_feasibility(z_mid)
        if feasible:
            z_low = z_mid
            best_x = x
        else:
            z_high = z_mid
    return z_low, best_x

# Initial range for z
z_low = 0
z_high = 1

# Perform bisection to find the optimal z
optimal_z, optimal_x = bisection_z(z_low, z_high)

if optimal_x is not None:
    x1_opt, x2_opt = optimal_x
    print(f"Optimal z: {optimal_z}")
    print(f"Optimal x1: {x1_opt}")
    print(f"Optimal x2: {x2_opt}")
else:
    print("No feasible solution found.")


Optimal z: 0.0030717849731445312
Optimal x1: 1.0
Optimal x2: 0.28215028992499014
